In [1]:
import pandas as pd

#### Importando os Dados

In [2]:
df_final_data_sumulas = pd.read_excel("data/final_data_original.xlsx", sheet_name="Sumulas", index_col=None)
df_final_data_atletas = pd.read_excel("atletas.xlsx", index_col=None)
df_final_data_competicoes = pd.read_excel("data/final_data_original.xlsx", sheet_name="Campeonatos", index_col=None)
df_sumulas_novas = pd.read_csv("sumulas.csv")

In [3]:
df_final_data_sumulas.shape

(136618, 14)

#### Tirando Coluna Problemática

In [4]:
df_final_data_sumulas.drop(df_final_data_sumulas.columns[0], axis=1, inplace=True)
df_final_data_atletas.drop(df_final_data_atletas.columns[0], axis=1, inplace=True)
df_final_data_competicoes.drop(df_final_data_competicoes.columns[0], axis=1, inplace=True)

#### Verificando Colunas

In [5]:
df_final_data_sumulas.columns

Index(['ID Resultado', 'ID Competidor', 'Nome Competidor', 'ID Competição',
       'Classe Vela', 'Pontuação Regata', 'Descarte', 'Flotilha',
       'Posição Geral', 'Punição', 'Pontuação Total', 'Nett',
       'Nome Competição'],
      dtype='object')

In [6]:
df_final_data_atletas.columns

Index(['Nome Competidor', 'ID Competidor'], dtype='object')

In [7]:
df_final_data_competicoes.columns

Index(['ID Competição', 'Nome Competição'], dtype='object')

In [8]:
df_sumulas_novas.columns

Index(['Identificador', 'Nome Competição', 'Regata', 'Nome Competidor',
       'Pontuação Regata (Nha)', 'Classe Vela', 'Flag Name', 'Posição Geral',
       'ID Competição', 'Descarte', 'Flotilha', 'Punição', 'Total', 'Net',
       'Pontuação Regata'],
      dtype='object')

#### Competições

In [9]:
df_final_data_competicoes = df_final_data_competicoes.sort_values(by="ID Competição").reset_index(drop=True)
df_final_data_competicoes.head(50)

,ID Competição,Nome Competição
0,1,Central and South American Championship 2021
1,2,Central and South American Championship 2022
2,3,Central and South American Championship 2023
3,4,European Championship 2021
4,5,European Championship 2022
5,6,European Championship 2023
6,7,Semaine Olympique Francaise De Voile 2022
7,8,Semaine Olympique Francaise De Voile 2023
8,9,Trofeo S.A.R Princesa Sofia 2022
9,10,Trofeo S.A.R Princesa Sofia 2023


In [10]:
df_final_data_atletas = df_final_data_atletas.sort_values(by="ID Competidor").reset_index(drop=True)
#df_final_data_atletas.head(10)

#### Fornecendo ID

In [11]:
def retorna_ID_competidor(nome_atleta:str):
    """
    Retorna o ID do competidor com base no nome do atleta.

    Parâmetros:
    nome_atleta (str): O nome do atleta cujo ID do competidor será retornado.

    Retorna:
    str: O ID do competidor correspondente ao nome do atleta especificado.

    Exemplo:
    >>> retorna_ID_competidor("DOMINIK BUKSAK, SZYMON WIERZBICKI")
    'ID correspondente'
    """
    return df_final_data_atletas.loc[df_final_data_atletas['Nome Competidor'] == nome_atleta, 'ID Competidor'].iloc[0]

In [12]:
def retorna_ID_competicao(nome_competicao: str):
    """
    Retorna o ID da competição com base no nome da competição.

    Parâmetros:
    nome_competicao (str): O nome da competição cujo ID será retornado.

    Retorna:
    str: O ID da competição correspondente ao nome da competição especificada.

    Exemplo:
    >>> retorna_ID_competicao("Nome da Competição")
    'ID correspondente'
    """
    retorno =  df_final_data_competicoes.loc[df_final_data_competicoes['Nome Competição'] == nome_competicao, 'ID Competição']
    return retorno.iloc[0] if not retorno.empty else None


#### Adicionando ID nos Novos Dados

In [13]:
df_sumulas_novas["ID Competidor"] = df_sumulas_novas["Nome Competidor"].apply(retorna_ID_competidor)
df_sumulas_novas["ID Competição"] = df_sumulas_novas["Nome Competição"].apply(retorna_ID_competicao)

In [14]:
df_final_data_sumulas["ID Competidor"] = df_final_data_sumulas["Nome Competidor"].apply(retorna_ID_competidor)
df_final_data_sumulas["ID Competição"] = df_final_data_sumulas["Nome Competição"].apply(retorna_ID_competicao)

#### Tratando as Flotilhas

In [15]:
df_sumulas_novas["Flotilha"].value_counts()

Flotilha
GERAL         20900
MEDAL RACE      628
Name: count, dtype: int64

In [16]:
df_sumulas_novas[df_sumulas_novas.columns[6]].value_counts()

Flag Name
OVER      10161
Silver     1794
Gold       1728
Bronze      935
Name: count, dtype: int64

In [17]:
mask = df_sumulas_novas["Flag Name"].isin(["Gold", "Silver", "Bronze"])

mapeamento = {"Gold": "OURO", "Silver": "PRATA", "Bronze": "BRONZE"}

df_sumulas_novas.loc[mask, "Flotilha"] = df_sumulas_novas.loc[mask, "Flag Name"].map(mapeamento)

In [18]:
df_sumulas_novas["Flotilha"].value_counts()

Flotilha
GERAL         16553
PRATA          1794
OURO           1728
BRONZE          935
MEDAL RACE      518
Name: count, dtype: int64

#### Arrumando as Colunas

In [19]:
df_final_data_sumulas.columns

Index(['ID Resultado', 'ID Competidor', 'Nome Competidor', 'ID Competição',
       'Classe Vela', 'Pontuação Regata', 'Descarte', 'Flotilha',
       'Posição Geral', 'Punição', 'Pontuação Total', 'Nett',
       'Nome Competição'],
      dtype='object')

In [20]:
df_sumulas_novas.columns
df_sumulas_novas["Total"].value_counts()

Total
0.0      6910
225.0     102
321.0     101
570.0     101
204.0      96
         ... 
32.0        7
43.0        7
111.0       7
51.0        7
75.0        7
Name: count, Length: 516, dtype: int64

In [26]:
df_sumulas_novas = df_sumulas_novas.rename(columns={'Total': 'Pontuação Total', "Net":"Nett"})
df_sumulas_novas.columns

Index(['Nome Competição', 'Nome Competidor', 'Classe Vela', 'Posição Geral',
       'ID Competição', 'Pontuação Regata', 'Descarte', 'Flotilha', 'Punição',
       'Pontuação Total', 'Nett', 'Pontuação Total', 'Nett', 'ID Competidor'],
      dtype='object')

In [22]:
colunas_extras = ["Identificador", "Regata", "Pontuação Regata (Nha)", "Flag Name"]
df_sumulas_novas = df_sumulas_novas.drop(columns=colunas_extras)
df_sumulas_novas.columns

Index(['Nome Competição', 'Nome Competidor', 'Classe Vela', 'Posição Geral',
       'ID Competição', 'Descarte', 'Flotilha', 'Punição', 'Pontuação Total',
       'Nett', 'Pontuação Regata', 'ID Competidor'],
      dtype='object')

In [23]:
ordem_colunas = ['Nome Competição', 'Nome Competidor', 'Classe Vela', 
                  'Posição Geral','ID Competição',
                  'Pontuação Regata', 'Descarte', 'Flotilha',
                  'Punição', 'Pontuação Total', 'Nett', 'ID Competidor'
                 ]

df_sumulas_novas = df_sumulas_novas.reindex(columns=ordem_colunas)
df_sumulas_novas.columns
df_sumulas_novas["Pontuação Total"]

,Pontuação Total,Pontuação Total
0,162.0,162.0
1,165.0,165.0
2,189.0,189.0
3,207.0,207.0
4,204.0,204.0
...,...,...
21523,0.0,0.0
21524,0.0,0.0
21525,0.0,0.0
21526,0.0,0.0


In [24]:
df_final_data_sumulas = df_final_data_sumulas.drop(columns=df_final_data_sumulas.columns[0])
df_final_data_sumulas.columns

Index(['ID Competidor', 'Nome Competidor', 'ID Competição', 'Classe Vela',
       'Pontuação Regata', 'Descarte', 'Flotilha', 'Posição Geral', 'Punição',
       'Pontuação Total', 'Nett', 'Nome Competição'],
      dtype='object')

#### Merge Dos Dados

In [25]:
df_data_final_atualizado = pd.concat([df_final_data_sumulas, df_sumulas_novas]).reset_index(drop=True)

InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [ ]:
df_data_final_atualizado["ID Resultado"] = df_data_final_atualizado.index + 1

In [ ]:
ordem_colunas = ['ID Resultado', 'ID Competidor', 'Nome Competidor', 'ID Competição',
                 'Classe Vela', 'Pontuação Regata', 'Descarte', 'Flotilha',
                 'Posição Geral', 'Punição', 'Pontuação Total', 'Nett',
                 'Nome Competição']

df_data_final_atualizado = df_data_final_atualizado.reindex(columns=ordem_colunas)
df_data_final_atualizado.columns

In [ ]:
df_data_final_atualizado.head()

In [ ]:
df_data_final_atualizado.tail()

#### Ajustes Nos Dados

In [ ]:
df_data_final_atualizado["Classe Vela"].value_counts()

In [ ]:
def ajusta_classe(valor):
    if valor.upper() == "KITE MASC.":
        return "FORMULA KITE MASC."
    elif valor.upper() == "KITE FEM.":
        return "FORMULA KITE FEM."
    elif str(valor) == "470.0":
        return "470"
    return str(valor).upper()

In [ ]:
df_data_final_atualizado["Classe Vela"] = df_data_final_atualizado["Classe Vela"].apply(ajusta_classe)

In [ ]:
df_data_final_atualizado["Classe Vela"].value_counts()

In [ ]:
df_data_final_atualizado["Flotilha"].value_counts()

In [ ]:
df_data_final_atualizado["Punição"].value_counts()

#### Salvando os Dados

In [ ]:
df_final_data_competicoes.to_excel("dados_finais_2024/competicoes.xlsx", index=False)
df_final_data_atletas.to_excel("dados_finais_2024/atletas.xlsx", index=False)
df_data_final_atualizado.to_excel("dados_finais_2024/sumulas.xlsx", index=False)

In [ ]:
with pd.ExcelWriter('dados_finais_2024/final_data.xlsx') as writer:
    # Salvando cada DataFrame em uma planilha diferente
    df_data_final_atualizado.to_excel(writer, sheet_name='Súmulas', index=False)
    df_final_data_atletas.to_excel(writer, sheet_name='Atletas', index=False)
    df_final_data_competicoes.to_excel(writer, sheet_name='Competições', index=False)